In [ ]:
# MQTT PUBLISH 測試應用程式

這是一個測試用的 MQTT 發布應用程式，用於發送感測器數據到 MQTT Broker。

## 功能
- 連接到本地 MQTT Broker
- 發布溫度、濕度、電燈狀態等測試數據
- 支援批次發送多筆數據
- 顯示發送狀態和結果


In [1]:
"""
MQTT PUBLISH 測試應用程式
匯入必要的套件
"""
import paho.mqtt.client as mqtt
import json
import time
from datetime import datetime
import random

print("✅ 套件匯入成功！")


✅ 套件匯入成功！


In [2]:
# MQTT 配置
BROKER = "localhost"  # MQTT Broker 位址
PORT = 1883           # MQTT 預設埠號
TOPIC = "客廳/感測器"  # 發布主題
CLIENT_ID = "jupyter_test_publisher"  # 客戶端 ID

print(f"📡 MQTT 設定：")
print(f"   Broker: {BROKER}:{PORT}")
print(f"   Topic: {TOPIC}")
print(f"   Client ID: {CLIENT_ID}")


📡 MQTT 設定：
   Broker: localhost:1883
   Topic: 客廳/感測器
   Client ID: jupyter_test_publisher


In [3]:
# 定義回調函數
def on_connect(client, userdata, flags, reason_code, properties):
    """連線成功時的回調"""
    if reason_code == 0:
        print("✅ 成功連接到 MQTT Broker")
    else:
        print(f"❌ 連線失敗，錯誤碼: {reason_code}")

def on_publish(client, userdata, mid, reason_code=None, properties=None):
    """訊息發布成功時的回調"""
    print(f"📤 訊息已發布 (ID: {mid})")

def on_disconnect(client, userdata, flags, reason_code, properties):
    """斷線時的回調"""
    if reason_code == 0:
        print("👋 已正常斷開連線")
    else:
        print(f"⚠️ 意外斷線，錯誤碼: {reason_code}")

print("✅ 回調函數定義完成")


✅ 回調函數定義完成


In [4]:
# 建立 MQTT 客戶端並連線
try:
    # 建立客戶端（使用 API Version 2）
    client = mqtt.Client(
        callback_api_version=mqtt.CallbackAPIVersion.VERSION2,
        client_id=CLIENT_ID
    )
    
    # 設定回調函數
    client.on_connect = on_connect
    client.on_publish = on_publish
    client.on_disconnect = on_disconnect
    
    # 連線到 Broker
    print(f"🔌 正在連接到 {BROKER}:{PORT}...")
    client.connect(BROKER, PORT, 60)
    
    # 啟動網路迴圈（非阻塞模式）
    client.loop_start()
    
    # 等待連線建立
    time.sleep(1)
    
except Exception as e:
    print(f"❌ 連線失敗: {e}")
    import traceback
    traceback.print_exc()


🔌 正在連接到 localhost:1883...
✅ 成功連接到 MQTT Broker


In [5]:
# 生成測試數據的函數
def generate_sensor_data(message_id=1):
    """
    生成模擬的感測器數據
    
    Args:
        message_id: 訊息編號
        
    Returns:
        dict: 包含感測器數據的字典
    """
    data = {
        "temperature": round(random.uniform(18, 30), 2),  # 溫度: 18-30°C
        "humidity": round(random.uniform(40, 80), 2),      # 濕度: 40-80%
        "light_status": random.choice(["開", "關"]),       # 電燈狀態
        "timestamp": datetime.now().isoformat(),           # 時間戳記
        "device": "Jupyter 測試裝置",                      # 裝置名稱
        "message_id": message_id                           # 訊息 ID
    }
    return data

# 測試生成一筆數據
test_data = generate_sensor_data()
print("📊 測試數據範例：")
print(json.dumps(test_data, indent=2, ensure_ascii=False))


📊 測試數據範例：
{
  "temperature": 25.46,
  "humidity": 66.48,
  "light_status": "開",
  "timestamp": "2025-12-12T22:32:59.445857",
  "device": "Jupyter 測試裝置",
  "message_id": 1
}


In [6]:
# 發布單筆測試數據
def publish_single_message():
    """發布一筆測試訊息"""
    try:
        # 生成數據
        data = generate_sensor_data()
        
        # 轉換為 JSON 字串
        json_data = json.dumps(data, ensure_ascii=False)
        
        # 發布訊息
        result = client.publish(TOPIC, json_data, qos=1)
        
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
            print(f"✅ 發布成功！")
            print(f"   溫度: {data['temperature']}°C")
            print(f"   濕度: {data['humidity']}%")
            print(f"   電燈: {data['light_status']}")
            print(f"   時間: {data['timestamp']}")
        else:
            print(f"❌ 發布失敗，錯誤碼: {result.rc}")
            
        return result
        
    except Exception as e:
        print(f"❌ 發生錯誤: {e}")
        return None

# 執行發布
print("=" * 60)
print("📤 發布單筆測試數據")
print("=" * 60)
publish_single_message()


📤 發布單筆測試數據
✅ 發布成功！
   溫度: 20.82°C
   濕度: 74.8%
   電燈: 開
   時間: 2025-12-12T22:33:34.424115
📤 訊息已發布 (ID: 1)


In [7]:
# 批次發布多筆測試數據
def publish_multiple_messages(count=5, interval=1):
    """
    批次發布多筆測試訊息
    
    Args:
        count: 發布數量
        interval: 間隔時間（秒）
    """
    print("=" * 60)
    print(f"📤 批次發布 {count} 筆測試數據（間隔 {interval} 秒）")
    print("=" * 60)
    
    success_count = 0
    fail_count = 0
    
    for i in range(count):
        try:
            # 生成數據
            data = generate_sensor_data(message_id=i+1)
            json_data = json.dumps(data, ensure_ascii=False)
            
            # 發布訊息
            result = client.publish(TOPIC, json_data, qos=1)
            
            if result.rc == mqtt.MQTT_ERR_SUCCESS:
                success_count += 1
                print(f"[{i+1}/{count}] ✅ 溫度: {data['temperature']}°C, "
                      f"濕度: {data['humidity']}%, 電燈: {data['light_status']}")
            else:
                fail_count += 1
                print(f"[{i+1}/{count}] ❌ 發布失敗")
            
            # 等待間隔時間（最後一筆不用等）
            if i < count - 1:
                time.sleep(interval)
                
        except Exception as e:
            fail_count += 1
            print(f"[{i+1}/{count}] ❌ 錯誤: {e}")
    
    # 顯示統計
    print("=" * 60)
    print(f"📊 發布統計：")
    print(f"   成功: {success_count} 筆")
    print(f"   失敗: {fail_count} 筆")
    print(f"   總計: {count} 筆")
    print("=" * 60)

# 執行批次發布（發布 5 筆，間隔 1 秒）
publish_multiple_messages(count=5, interval=1)


📤 批次發布 5 筆測試數據（間隔 1 秒）
[1/5] ✅ 溫度: 23.02°C, 濕度: 54.09%, 電燈: 開
📤 訊息已發布 (ID: 2)
[2/5] ✅ 溫度: 18.34°C, 濕度: 46.53%, 電燈: 開
📤 訊息已發布 (ID: 3)
[3/5] ✅ 溫度: 22.15°C, 濕度: 47.03%, 電燈: 開
📤 訊息已發布 (ID: 4)
[4/5] ✅ 溫度: 24.44°C, 濕度: 43.48%, 電燈: 關
📤 訊息已發布 (ID: 5)
[5/5] ✅ 溫度: 25.44°C, 濕度: 58.23%, 電燈: 開📤 訊息已發布 (ID: 6)

📊 發布統計：
   成功: 5 筆
   失敗: 0 筆
   總計: 5 筆


In [8]:
# 自訂數據發布
def publish_custom_data(temperature, humidity, light_status):
    """
    發布自訂的感測器數據
    
    Args:
        temperature: 溫度值
        humidity: 濕度值
        light_status: 電燈狀態（"開" 或 "關"）
    """
    try:
        data = {
            "temperature": temperature,
            "humidity": humidity,
            "light_status": light_status,
            "timestamp": datetime.now().isoformat(),
            "device": "Jupyter 自訂數據",
            "message_id": int(time.time())
        }
        
        json_data = json.dumps(data, ensure_ascii=False)
        result = client.publish(TOPIC, json_data, qos=1)
        
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
            print(f"✅ 自訂數據發布成功！")
            print(f"   {json.dumps(data, indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ 發布失敗")
            
        return result
        
    except Exception as e:
        print(f"❌ 錯誤: {e}")
        return None

# 範例：發布自訂數據
print("=" * 60)
print("📤 發布自訂數據")
print("=" * 60)
publish_custom_data(temperature=25.5, humidity=65.0, light_status="開")


📤 發布自訂數據
✅ 自訂數據發布成功！
   {
  "temperature": 25.5,
  "humidity": 65.0,
  "light_status": "開",
  "timestamp": "2025-12-12T22:34:04.712940",
  "device": "Jupyter 自訂數據",
  "message_id": 1765550044
}


📤 訊息已發布 (ID: 7)


In [9]:
# 清理：關閉 MQTT 連線
def cleanup():
    """關閉 MQTT 連線並清理資源"""
    try:
        print("🧹 正在清理資源...")
        client.loop_stop()
        client.disconnect()
        print("✅ MQTT 連線已關閉")
    except Exception as e:
        print(f"⚠️ 清理時發生錯誤: {e}")

# 注意：在 Jupyter 中，這個 cell 要手動執行來關閉連線
# 如果要繼續使用 MQTT，不要執行這個 cell
print("⚠️ 執行此 cell 將關閉 MQTT 連線")
print("💡 如果還要繼續測試，請不要執行這個 cell")


⚠️ 執行此 cell 將關閉 MQTT 連線
💡 如果還要繼續測試，請不要執行這個 cell
